In [ ]:
# Загрузка библиотеки OpenAI-Whisper
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

In [ ]:
import os
import numpy as np
import torch
import pandas as pd
import whisper

# Для создания прогресс-баров
from tqdm.notebook import tqdm 

# Переключение на GPU, если GPU доступно
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Инициализация модели Whisper Large
model = whisper.load_model("large")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters." )

# Настройка опций: язык — русский, временные метки — нужны.
options = whisper.DecodingOptions(language="ru", without_timestamps=False)

In [ ]:
from whisper.utils import get_writer
from text_processing import import_text, export_text
from functions import change_name, change_folder_name

for (prompt, item) in [ # Указываем, какой файл транскрибировать и с каким initial_prompt 
                       ('prompts/Sep 1 (Начало).txt', 'audio (dry)/Гришунин 1 (Yandex, 01.09.2023).mp3')
                       ('prompts/Sep 8 (Спиноза).txt', 'audio (dry)/Sep 8, Гришунин 2.mp3')
                       ('prompts/Sep 15 (Беркли).txt', 'audio (dry)/Sep 15, Гришунин 3.mp3')
                       ('prompts/Sep 22 (Кант).txt', 'audio (dry)/Sep 22, Гришунин 4.mp3')
                       ('prompts/Oct 6 (Кант).txt', 'audio (dry)/Oct 6, Гришунин 6.mp3')
                       ('prompts/Oct 13 (Фихте).txt', 'audio (dry)/Oct 13, Гришунин 7.mp3')
                       ('prompts/Oct 20 (Гегель).txt', 'audio (dry)/Oct 20, Гришунин 8.mp3')
                       ('prompts/Oct 27 (Фейербах).txt', 'audio (dry)/Oct 27, Гришунин 9.mp3')
                       ('prompts/Nov 3 (Энгельс).txt', 'audio (dry)/Nov 3, Гришунин 10.mp3')
                       ('prompts/Nov 10 (Бергсон).txt', 'audio (dry)/Nov 10, Гришунин 11.mp3')
                       ('prompts/Nov 17 (Чижевский).txt', 'audio (dry)/Nov 17, Гришунин 12.mp3')
                       ('prompts/Nov 24 (Конт).txt', 'audio (dry)/Nov 24, Гришунин 13.mp3')
                       ('prompts/Sep 29 (Фихте).txt', 'audio (dry)/Sep 29, Гришунин 5.mp3')
                       ('prompts/Dec 1 (Мах).txt', 'audio (dry)/Nov 10, Гришунин 11.mp3 audio (dry)/Dec 1, Гришунин 14.mp3')
                       ]:
    
    # Импортируем указанный prompt
    prompt_transcribe = import_text(prompt)

    # Транскрибируем файл
    result = model.transcribe(item, # Файл для обработки
                              initial_prompt=prompt_transcribe, # Входной prompt для fine-tuning'а словаря
                              verbose=True, # Выводим результаты на экран сразу по ходу работы
                              condition_on_previous_text=False, # Распознаваемый текст НЕ воспринимаем как продолжение prompt
                              temperature=.1, # Температура — мера "креативности" модели
                              **{"language": 'Russian'} # Дополнительные аргументы для транскрибации
                                )
    
    # Указываем имена выходных файлов (для txt и srt файлов соответственно)
    output_txt = change_folder_name(change_name(item, extension='.txt'), 'transcript_txt')
    output_srt = change_folder_name(change_name(item, extension=''), 'transcript_srt')
    
    # Запись текста без тайм-кодов
    export_text(result['text'], change_name(item, extension='.txt'))

    # Задаём текущую папку и сохраняем субтитры в формате SRT (субтитры с таймкодами)
    writer = get_writer("srt", ".")  
    writer(result, change_name(item, extension=''), {})